# 02d Emparejamiento

El "emparejamiento" (matching) es una nueva estructura de Python introducida en la versión 3.10. Comprueba que estás usando una versión de Python adecuada:

In [1]:
import sys
sys.version

'3.10.8 (main, Oct 12 2022, 00:00:00) [GCC 12.2.1 20220819 (Red Hat 12.2.1-2)]'

## Emparejamiento de constantes

En su forma más sencilla, el emparejamiento funciona como una condicional múltiple, similar a `switch` en C/C++. EL valor de `match` se compara con una o varias constantes:

In [2]:
x = 5

match x:
    case 3:
        print("x vale 3")
    case 7:
        print("x vale 7")
    case _:
        print("x vale otro valor")

x vale otro valor


La cláusula `case _` es el equivalente a un `else`, emapreja con cualquier valor.

Podemos combinar varias constantes con `|`, y el código se ejecutará si coincide con uno de ellos:

In [3]:
x = 5

match x:
    case 2 | 3 | 5 | 7:
        print("x es un número primo menor de 10")
    case _:
        print("No lo es")

x es un número primo menor de 10


Para tipos compuestos (incluyendo clases), podemos pedir que emparejen uno o varios de los elementos. Para los elementos que ignoramos, hay que usar `_`:

In [9]:
def posicion_punto(x):
    match x:
        case (0, 0):
            print("El punto es el origen")
        case (0, _):
            print("El punto está en el eje Y")
        case (_, 0):
            print("El punto está en el eje X")
        case (_, _):
            print("El punto no está en los ejes")
        case _:
            raise TypeError

In [10]:
posicion_punto((3, 0))
posicion_punto((0, -7))
posicion_punto((1, 8))
posicion_punto((0,0))
posicion_punto("a")

El punto está en el eje X
El punto está en el eje Y
El punto no está en los ejes
El punto es el origen


TypeError: 

## Emparejamiento de tipo

`match` puede emparejar con valores que sean de un tipo determinado:

In [1]:
def primer_elemento(x):
    """Si x es un entero, lo devuelve, y si es una lista, devuelve el primer elemento"""
    match x:
        case float():
            return x
        case list():
            return x[0]
        case _:
            raise TypeError

In [2]:
print(primer_elemento(5.2))
print(primer_elemento([-3.7, 4.5]))

5.2
-3.7


Podemos ser más específicos, y pedir que el primer elemento de la lista sea un `float`:

In [4]:
def primer_elemento(x):
    """Si x es un float, lo devuelve, y si es una lista, devuelve el primer elemento"""
    match x:
        case float():
            return x
        case [float(), *_]:
            return x[0]
        case _:
            raise TypeError

El asterisco delante de `_` es necesario para que empareje con listas de cualquier longitud (mayor o igual que 1)

In [5]:
print(primer_elemento(5.2))
print(primer_elemento([-3.7, 4.5, -1.2]))
print(primer_elemento([3.41,]))
print(primer_elemento([2, 7]))

5.2
-3.7
3.41


TypeError: 

O podemos pedir que el primer elemento sea un `float` o `int`:

In [7]:
def primer_elemento(x):
    """Si x es un float, lo devuelve, y si es una lista, devuelve el primer elemento"""
    match x:
        case float() | int():
            return x
        case [float() | int(), *_]:
            return x[0]
        case _:
            raise TypeError

In [8]:
print(primer_elemento(5.2))
print(primer_elemento([-3.7, 4.5, -1.2]))
print(primer_elemento([3.41,]))
print(primer_elemento([2, 7]))

5.2
-3.7
3.41
2


## Emparejamiento con asignación

La ventaja de `match` es que es capaz de asignar variables al emparejar. En el ejemplo anterior, en vez de devolver `x[0]`, podemos asignar el valor del primer elemento a una variable `y`, y el resto de la lista a una variable `resto`:

In [11]:
def primer_elemento(x):
    """Si x es un float, lo devuelve, y si es una lista, devuelve el primer elemento"""
    match x:
        case float() | int():
            return x
        case [float(y) | int(y), *resto]:
            print(f"Valores descartados: {resto}")
            return y
        case _:
            raise TypeError

In [12]:
print(primer_elemento(5.2))
print(primer_elemento([-3.7, 4.5, -1.2]))
print(primer_elemento([3.41,]))
print(primer_elemento([2, 7]))

5.2
Valores descartados: [4.5, -1.2]
-3.7
Valores descartados: []
3.41
Valores descartados: [7]
2


Veamos un ejemplo más complicado, donde usamos `match` para capturar los valores de dos coeficientes de Wilson en concreto:

In [28]:
from wilson import Wilson, wcxf

def extrae_coefs(w):
    match w:
        case Wilson():
            match w.wc:
                case wcxf.WC(values={'lq1_2223': lq1, 'lq3_2223': lq3}, eft="SMEFT", basis="Warsaw"):
                    return (lq1, lq3)
                case wcxf.WC(eft="SMEFT", basis="Warsaw"):
                    raise KeyError("No existen los coeficientes lq1_2223 o lq3_2223")
                case wcxf.WC(eft="SMEFT"):
                    raise ValueError("Los coeficientes no están en la base Warsaw")
                case _:
                    raise ValueError("Los coeficientes no pertenecen al SMEFT")
        case _:
            raise TypeError("El objeto no es una instancia de Wilson")

Como ves, también es posible emparejar miembros de una clase. En general, hay que especificar el nombre del miembro antes del signo igual.

La estructura interna de la clase `Wilson` es un poco enrevesada, porque en vez de guardar los coeficientes y la teoría efectiva como miembros, los almacena como un objeto de la clase `wcxf.WC`, y por eso estamos usando dos `match` anidados. 


Comprobamos que el `match` funciona correctamente con un objeto `Wilson` que contiene los coeficientes que buscamos:

In [29]:
extrae_coefs(Wilson({'lq1_2223': 1e-7, 'lq3_2223': 1e-7}, scale = 1000, eft="SMEFT", basis="Warsaw"))

(1e-07, 1e-07)

In [30]:
extrae_coefs(Wilson({'lq1_2223': 1e-7, 'lq3_2223': 1e-7, 'lq1_1123': 1e-7}, scale = 1000, eft="SMEFT", basis="Warsaw"))

(1e-07, 1e-07)

y que si no los contiene, creamos las excepciones adecuadas:

In [34]:
extrae_coefs(Wilson({'lq1_2223': 1e-7, 'lq3_1123': 1e-7}, scale = 1000, eft="SMEFT", basis="Warsaw"))

KeyError: 'No existen los coeficientes lq1_2223 o lq3_2223'

In [32]:
extrae_coefs(Wilson({'lq1_2223': 1e-7, 'lq3_2223': 1e-7}, scale = 1000, eft="SMEFT", basis="Warsaw up"))

ValueError: Los coeficientes no están en la base Warsaw

In [33]:
extrae_coefs(Wilson({'C9_bsmumu': -0.6}, scale=4.8, eft="WET", basis="flavio"))

ValueError: Los coeficientes no pertenecen al SMEFT

Al crear una clase, es posible especificar que (algunos) miembros se puedan emparejar sin necesidad de escribir el nombre del miembro, sino por el orden en el que aparecen. Para ello, hay que añadir a la clase la tupla `__match_args__`:

In [36]:
class Cuadrivector:
    __match_args__ = ("t", "x", "y", "z")
    def __init__(self, t, x, y, z):
        self.t = float(t)
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

p = Cuadrivector(1, 0, 0, 0)

match p:
    case Cuadrivector(m, 0.0, 0.0, 0.0):
        print(f"Objeto de masa {m} en reposo")
    case Cuadrivector(p0, px, py, pz):
        m = (p0**2-px**2-py**2-pz**2)**0.5
        print(f"Objeto de masa {m} en movimiento")
    case _:
        raise TypeError("No es un cuadrivector")
    

Objeto de masa 1.0 en reposo
